# 2. Process each row and feed it for ChapGPT to process

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Load the API key from config file
load_dotenv("./config/api_key.env")
# Fetch the API key from the environment
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [5]:
# Debug
df = pd.read_csv('./PostHistoryFirstVersion.csv')
for index, row in df.iterrows():
    print(f"Title of the {index+1} post:")
    print(row['Title'])
    print()
    
    print(f"Body of the {index+1} post:")
    print(row['Body'])
    print()
    
    print(f"Tag of the {index+1} post:")
    print(row['Tags'])
    print()

Title of the 1 post:
Make a div fill the remaining screen space

Body of the 1 post:
I am currently working on a web application, where I want the content to fill the entire screen. I have a header `div` and a content `div`. At the moment I am using a table for the layout like so:

CSS:

    #page {
      height: 100%;
    }
    #tdcontent {
      height: 100%;
    }

HTML:

    <table id="page">
      <tr><td id="tdheader">
        <div id="header">...</div>
      </td></tr>
      <tr><td id="tdcontent>
        <div id="content">...</div>
      </td>
    </table>

Which works how I like. The table remains at 100%, and the `tdcontent` element is always less than 100%. Is there a way to achieve the same effect without using the `table`?

Tag of the 1 post:
<table><css><html>

Title of the 2 post:
Compare 2 dates with JavaScript

Body of the 2 post:
can some one suggest a way to compare values of 2 dates greater then, less then and not in the passed using javascript. They values will be 

In [ ]:
# Load the dataset
df = pd.read_csv('./PostHistoryFirstVersion.csv')

# List to store the improved version of title, body and tags
new_titles = []
new_bodies = []
new_tags = []

# Loop through each row of the df
for index, row in df.iterrows():
    # Construct the instruction for dev role
    instructions = f"""
    # Identity
    You are a developer assistant that improves Stack Overflow questions.
    
    # Instructions
    * Rewrite the Title to be more clear and professional.
    * Rewrite the Body to be clearer and more detailed if needed.
    * Add or Adjust Tags if necessary.
    
    * Format your response as:
    _ Title: your improved title\n
    _ Body: your improved body\n
    _ Tags: your improved tags separated <>, like this <tag1><tag2><tag3>\n
    
    """
    
    # Construct the input for the user role
    prompt = f"""
    Could you rewrite this Stackoverflow question given 3 information below:
    _ Original Title:
    {row['Title']}
    
    _ Original Body:
    {row['Body']}
    
    _ Original Tags:
    {row['Tags']}
    """
    # Prompt using Chat Completion API
    completion = client.chat.completions.create(
        model="gpt-4.1-nano",  
        
        messages=[
            {
                "role": "system",
                "content": instructions
            },
            {
                "role": "developer",
                "content": "Response like you are a senior developer writing for a question on Stack Overflow. With clear and concise Title, Body, Tags"
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        
        n=1, 
        temperature=0.3,
        max_tokens=1000
    )
    
    # Parse the response
    reply = completion.choices[0].message.content
    
    # Very basic parsing
    title_only = reply.split('Title:')[1].split('Body:')[0].strip()
    body_only = reply.split('Body:')[1].split('Tags:')[0].strip()
    tags_only = reply.split('Tags:')[1].strip()
    
    new_titles.append(title_only)
    new_bodies.append(body_only)
    new_tags.append(tags_only)

In [7]:
# print(new_titles)
# print(new_bodies)
# print(new_tags)

# Add new columns to the DataFrame
df['Improved_Title'] = new_titles
df['Improved_Body'] = new_bodies
df['Improved_Tags'] = new_tags

# Save to new CSV
df.to_csv('ImprovedQuestions.csv', index=False)

In [8]:
from IPython.display import display
display(df)

,PostId,Title,Body,Tags,Improved_Title,Improved_Body,Improved_Tags
0,90178,Make a div fill the remaining screen space,"I am currently working on a web application, w...",<table><css><html>,How to Make a Div Fill Remaining Screen Space ...,I'm developing a web application where I want ...,<css><html><layout><flexbox><grid>
1,492994,Compare 2 dates with JavaScript,can some one suggest a way to compare values o...,<date><compare>,How to Compare Two Dates in JavaScript for Gre...,I'm working on a JavaScript project where I ne...,<javascript><date><comparison><user-input>
2,500504,"JavaScript ""For ...in"" with Arrays","I've been told not to use ""for...in"" with Arra...",<javascript>,"Why is it problematic to use ""for...in"" loops ...","I've heard that using ""for...in"" loops with Ar...",<javascript><iteration><arrays>
3,1149703,Stacktrace to string in Java,Easiest way to convert the result of Throwable...,<stack-trace><java><tostring>,How to Convert Throwable StackTrace to String ...,I'm looking for the simplest way to convert th...,<java><stack-trace><exception>
4,2709821,python 'self' explained,What is the purpose of the 'self' word in pyth...,<python><self><explain>,Clarification on the Use of 'self' in Python C...,I'm trying to understand the purpose of the 's...,<python><self><class-methods><object-oriented-...
5,3106452,java servlet instantiation and session variables,"Out of curiosity and naivety:\r\n\r\nSuppose, ...",<servlets>,Understanding Servlet Instantiation and Sessio...,I am working with a Java web server that hosts...,<servlets><session-management><servlet-instances>
6,3289601,nulls object in Python?,How do I refer to the null object in Python?,<python><null>,How to Represent and Use Null Values in Python,I'm trying to understand how to handle null or...,<python><null><null-value>
7,5844672,Delete an element from a dictionary,Is there a way to delete an element from a dic...,<python><dictionary><remove>,How to Remove an Element from a Python Diction...,I'm looking for a way to remove an element fro...,<python><dictionary><remove><copy>
8,9483757,END in git bash,I'm trying to learn git with the help of git i...,<git>,How to Exit the 'END' Prompt in Git Bash When ...,_\nI'm following the Git Immersion tutorial (h...,<git><bash><pager><less>
9,10123953,Sort Javascript Object Array By Date,Say I have an array of a few objects:\r\n\r\n ...,<javascript><datetime>,How to Sort an Array of JavaScript Objects by ...,"I have an array of objects in JavaScript, each...",<javascript><date><sorting><array>
